In [1]:
from __future__ import unicode_literals

import json
from time import sleep
from bs4 import BeautifulSoup

In [2]:

from kafka import KafkaConsumer, KafkaProducer

import requests
# -*- coding: utf-8 -*-


In [3]:
def fetch_raw(recipe_url):
    html = None
    print('Processing..{}'.format(recipe_url))
    try:
        r = requests.get(recipe_url, headers=headers)
        if r.status_code == 200:
            html = r.text
             
    except Exception as ex:
        print('Exception while accessing raw html')
        print(str(ex))
    finally:
        return html.strip()


def get_recipes():
    recipies = []
    salad_url = 'https://www.allrecipes.com/recipes/156/bread/'
    url = 'https://www.allrecipes.com/recipes/156/bread/'
    url = 'https://www.allrecipes.com/recipes/23014/world-cuisine/asian/thai/desserts'
    print('Accessing list')

    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            html = r.text
            #soup = BeautifulSoup(html, 'lxml')
            soup = BeautifulSoup(html, 'html.parser')
            #soup = BeautifulSoup(r.content, features="html")
            links = soup.select('.card__titleLink')
             
            idx = 0
            print('total',len(links))
            for link in links:

                sleep(2)
                #print(link)
                recipe = fetch_raw(link['href'])
                #print(recipe)
                recipies.append(recipe)
                
                #print(link['href'])
                idx += 1
                if idx > 3:
                    break
    except Exception as ex:
        print('Exception in get_recipes')
        print(str(ex))
    finally:
        return recipies

In [4]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        #print(key)
        #print(value)
        key_bytes =  bytes(key, encoding='utf-8')   #.encode('utf-8')
        value_bytes = bytes(value, encoding='utf-8') #encode('utf-8')
        
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        #print(value_bytes)
        producer_instance.flush()
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))


def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer

In [5]:
if __name__ == '__main__':
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
        'Pragma': 'no-cache'
    }
    all_recipes = get_recipes()
    #print(all_receipes)
    
    i = 0
    if len(all_recipes) > 0:
        kafka_producer = connect_kafka_producer()
        for recipe in all_recipes:
            i = i+1
            publish_message(kafka_producer, 'raw_recipes', 'raw', recipe.strip())
            if i > 3:
                break
        if kafka_producer is not None:
            kafka_producer.close()

Accessing list


/home/hadoop/.local/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


total 24
Processing..https://www.allrecipes.com/recipe/150313/thai-sweet-sticky-rice-with-mango-khao-neeo-mamuang/
Processing..https://www.allrecipes.com/recipe/150313/thai-sweet-sticky-rice-with-mango-khao-neeo-mamuang/
Processing..https://www.allrecipes.com/recipe/219490/sweet-sticky-rice-with-mangoes/
Processing..https://www.allrecipes.com/recipe/219490/sweet-sticky-rice-with-mangoes/
Message published successfully.
Message published successfully.
Message published successfully.
Message published successfully.
